# Проект "Выбор локации для скважины"

## Введение

Вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину. 
Шаги для выбора локации обычно такие:
 - В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
 - Строят модель для предсказания объёма запасов в новых скважинах;
 - Выбирают скважины с самыми высокими оценками значений;
 - Определяют регион с максимальной суммарной прибылью отобранных скважин.
 
Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

## Загрузка данных

In [1]:
!pip install -q phik


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, precision_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from phik import phik_matrix
from phik.report import plot_correlation_matrix
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('max_colwidth', -1)
from sklearn.svm import SVC, SVR
matplotlib_axes_logger.setLevel('ERROR')
RANDOM_STATE = 42
TEST_SIZE = 0.25
sns.set()

In [3]:
df_geo_data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
display(df_geo_data_0.head(5))
df_geo_data_0.info()

df_geo_data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
display(df_geo_data_1.head(5))
df_geo_data_1.info()

df_geo_data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')
display(df_geo_data_2.head(5))
df_geo_data_2.info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


## Предобработка данных

## Исследовательский анализ данных

## Подготовка данных

## Обучение модели для каждого региона

### Разбиение данных на обучающую и валидационную выборки в соотношении 75:25

### Обучение модели и генерация предсказаний на валидационной выборке

### Сохранение предсказаний и правильных ответов на валидационной выборке

### Вывод среднего запаса предсказанного сырья и RMSE модели

### Анализ результатов

## Рассчет прибыли

### Все ключевые значения для расчётов сохранить в отдельных переменных

### Рассчет достаточного объёма сырья для безубыточной разработки новой скважины

### Выводы

## Разработка функции для расчёта прибыли по выбранным скважинам и предсказаниям модели

### Выбор скважины с максимальными значениями предсказаний

### Суммирование целевого значения объёма сырья, соответствующее этим предсказаниям

### Рассчет прибыли для полученного объёма сырья

## Рассчет рисков и прибыли для каждого региона

### Применение техники Bootstrap с 1000 выборок, чтобы найти распределение прибыли

### Определение средней прибыли, 95%-й доверительного интервала и риска убытков

### Выводы по определению самого прибыльного региона

## Общие выводы